In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

#data from user
city = input("The city that you want to exctract data(in Italian): ").strip()
num_pages = int(input("how many pages would you want to exctract: "))

# browser setting
options = Options()
options.add_experimental_option("detach", True)  #it keeps browser open to skip the captcha
options.add_argument("--disable-blink-features=AutomationControlled")

# run driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# save all the data in a file
with open("all_pages.html", "w", encoding="utf-8") as file:
    for page in range(1, num_pages + 1):
        if page == 1:
            url = f"https://www.immobiliare.it/vendita-case/{city}/"
        else:
            url = f"https://www.immobiliare.it/vendita-case/{city}/?pag={page}"
        
        print(f"reciving data: {url}")
        
        driver.get(url)
        input("please skip captcha then press enter")  #waiting for captcha

        file.write(f"<!-- Page {page} -->\n")
        file.write(driver.page_source)
        file.write("\n\n")
    
print("✅ all the pages saved in a file")
driver.quit()


In [ ]:
import csv
from bs4 import BeautifulSoup

# read html file
with open("all_pages.html", "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

# extract prices
prices = [price.get_text(strip=True) for price in soup.find_all("div", class_="in-listingCardPrice")]

#extract size of the houses
sizes = [size.get_text(strip=True) for size in soup.find_all("div", class_="in-listingCardFeatureList__item") if "m²" in size.get_text()]

# number of rooms
rooms = [room.get_text(strip=True) for room in soup.find_all("div", class_="in-listingCardFeatureList__item") if "locali" in room.get_text()]

# save to csv file
# it would be used to train my ML model
with open("data.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Price (€)", "Size (m²)", "Rooms"])  #write headers
    for price, size, room in zip(prices, sizes, rooms):
        writer.writerow([price, size, room])

print("✅ Data successfully saved in 'data.csv'!")
